### 🎰 Calculadora de Probabilidades de Números de Lotería

Este script permite **consultar un número de lotería y analizar sus probabilidades históricas y teóricas**, basándose en el conjunto de datos procesado (`loteria_datos.pkl`).  
El análisis incluye **frecuencia histórica, probabilidad teórica, probabilidades de aproximaciones (centena, decena y reintegro)** y un **factor de frecuencia comparado con la media**.

---


In [1]:
import pandas as pd

# Cargar los datos
df = pd.read_pickle('loteria_datos.pkl')

In [2]:
def calcular_probabilidad(numero_input):
    """
    Calcula la probabilidad de que toque un número específico
    
    Args:
        numero_input: Número de lotería (string de 5 dígitos, ej: "00219")
    
    Returns:
        dict con las probabilidades calculadas
    """
    # Asegurar formato correcto (5 dígitos con ceros a la izquierda)
    numero = str(numero_input).zfill(5)
    
    # Datos del análisis
    total_sorteos = len(df)
    numeros_unicos = df['numero'].nunique()
    veces_salido = (df['numero'] == numero).sum()
    
    # Extraer partes del número
    centena = numero[-3:]  # Últimos 3 dígitos
    decena = numero[-2:]   # Últimos 2 dígitos
    unidad = numero[-1:]   # Último dígito
    
    # Filtrar por tipo de sorteo si se especifica
    df_filtrado = df[df['tipo'] != 'normal']
    
    # Conteos EXCLUYENDO premios mayores (para evitar doble conteo)
    veces_numero_exacto = (df_filtrado['numero'] == numero).sum()
    
    # Centena: todos los que terminan en centena EXCEPTO el número exacto
    veces_centena_total = df_filtrado['numero'].str[-3:].eq(centena).sum()
    veces_centena = veces_centena_total - veces_numero_exacto
    
    # Decena: todos los que terminan en decena EXCEPTO número exacto y centena
    veces_decena_total = df_filtrado['numero'].str[-2:].eq(decena).sum()
    veces_decena = veces_decena_total - veces_centena_total
    
    # Reintegro: todos los que terminan en unidad EXCEPTO número exacto, centena y decena
    veces_unidad_total = df_filtrado['numero'].str[-1:].eq(unidad).sum()
    veces_unidad = veces_unidad_total - veces_decena_total
    
    # 1. Probabilidad teórica (todos los números posibles: 00000-99999)
    prob_teorica = 1 / 100000
    prob_teorica_porcentaje = prob_teorica * 100
    
    # 2. Probabilidad basada en datos históricos
    prob_historica = veces_salido / total_sorteos if total_sorteos > 0 else 0
    prob_historica_porcentaje = prob_historica * 100
    
    # 3. Probabilidad basada en números únicos que han salido
    prob_numeros_existentes = veces_salido / numeros_unicos if numeros_unicos > 0 else 0
    prob_numeros_existentes_porcentaje = prob_numeros_existentes * 100
    
    # 4. Frecuencia relativa (comparado con la media)
    frecuencia_media = total_sorteos / numeros_unicos if numeros_unicos > 0 else 0
    factor_frecuencia = veces_salido / frecuencia_media if frecuencia_media > 0 else 0
    
    # 5. Probabilidades de aproximaciones (solo para especiales)
    total_filtrado = len(df_filtrado)
    
    prob_centena = veces_centena / total_filtrado if total_filtrado > 0 else 0
    prob_centena_porcentaje = prob_centena * 100
    
    prob_decena = veces_decena / total_filtrado if total_filtrado > 0 else 0
    prob_decena_porcentaje = prob_decena * 100
    
    prob_unidad = veces_unidad / total_filtrado if total_filtrado > 0 else 0
    prob_unidad_porcentaje = prob_unidad * 100
    
    # Probabilidad teórica de aproximaciones
    prob_teorica_centena = 1 / 1000  # Últimos 3 dígitos
    prob_teorica_decena = 1 / 100    # Últimos 2 dígitos
    prob_teorica_unidad = 1 / 10     # Último dígito
    
    return {
        'numero': numero,
        'veces_salido': veces_salido,
        'total_sorteos': total_sorteos,
        'numeros_unicos': numeros_unicos,
        'prob_teorica': prob_teorica,
        'prob_teorica_porcentaje': prob_teorica_porcentaje,
        'prob_historica': prob_historica,
        'prob_historica_porcentaje': prob_historica_porcentaje,
        'prob_numeros_existentes': prob_numeros_existentes,
        'prob_numeros_existentes_porcentaje': prob_numeros_existentes_porcentaje,
        'frecuencia_media': frecuencia_media,
        'factor_frecuencia': factor_frecuencia,
        'centena': centena,
        'decena': decena,
        'unidad': unidad,
        'veces_centena': veces_centena,
        'veces_decena': veces_decena,
        'veces_unidad': veces_unidad,
        'prob_centena': prob_centena,
        'prob_centena_porcentaje': prob_centena_porcentaje,
        'prob_decena': prob_decena,
        'prob_decena_porcentaje': prob_decena_porcentaje,
        'prob_unidad': prob_unidad,
        'prob_unidad_porcentaje': prob_unidad_porcentaje,
        'prob_teorica_centena': prob_teorica_centena,
        'prob_teorica_decena': prob_teorica_decena,
        'prob_teorica_unidad': prob_teorica_unidad
    }

In [ ]:
def mostrar_resultados(numero_input):
    """Muestra los resultados de forma legible"""
    resultado = calcular_probabilidad(numero_input)
    
    print("=" * 70)
    print(f"ANÁLISIS DE PROBABILIDAD PARA EL NÚMERO: {resultado['numero']}")
    print("=" * 70)
    
    print(f"\n📊 DATOS HISTÓRICOS:")
    print(f"   • El número {resultado['numero']} ha salido: {resultado['veces_salido']} veces")
    print(f"   • Total de sorteos analizados: {resultado['total_sorteos']}")
    print(f"   • Números únicos premiados: {resultado['numeros_unicos']}")
    
    print(f"\n🎲 PROBABILIDADES:")
    print(f"\n   1. Probabilidad TEÓRICA (real entre todos los españoles):")
    print(f"      → 1 entre 100,000 números posibles")
    print(f"      → {resultado['prob_teorica_porcentaje']:.5f}%")
    print(f"      → Odds: 1:{100000:,}")
    
    print(f"\n   2. Probabilidad HISTÓRICA (basada en los datos recogidos):")
    print(f"      → Basada en {resultado['total_sorteos']} sorteos")
    print(f"      → {resultado['prob_historica_porcentaje']:.5f}%")
    if resultado['veces_salido'] > 0:
        print(f"      → Sale aproximadamente 1 cada {resultado['total_sorteos']//resultado['veces_salido']} sorteos")
    
    print(f"\n   3. Probabilidad entre NÚMEROS QUE HAN SALIDO:")
    print(f"      → Solo considerando los {resultado['numeros_unicos']} números que han sido premiados")
    print(f"      → {resultado['prob_numeros_existentes_porcentaje']:.5f}%")

    print(f"\n   📍 CENTENA (terminados en {resultado['centena']}):")
    print(f"      • Ha salido: {resultado['veces_centena']} veces")
    print(f"      • Probabilidad teórica: {resultado['prob_teorica_centena']*100:.3f}% (1 en 1,000)")
    print(f"      • Probabilidad histórica: {resultado['prob_centena_porcentaje']:.3f}%")
        
    print(f"\n   📍 DECENA (terminados en {resultado['decena']}):")
    print(f"      • Ha salido: {resultado['veces_decena']} veces")
    print(f"      • Probabilidad teórica: {resultado['prob_teorica_decena']*100:.2f}% (1 en 100)")
    print(f"      • Probabilidad histórica: {resultado['prob_decena_porcentaje']:.3f}%")
        
    print(f"\n   📍 REINTEGRO (terminados en {resultado['unidad']}):")
    print(f"      • Ha salido: {resultado['veces_unidad']} veces")
    print(f"      • Probabilidad teórica: {resultado['prob_teorica_unidad']*100:.1f}% (1 en 10)")
    print(f"      • Probabilidad histórica: {resultado['prob_unidad_porcentaje']:.3f}%")
    
    print(f"\n📈 ANÁLISIS DE FRECUENCIA:")
    print(f"   • Frecuencia media por número: {resultado['frecuencia_media']:.2f} veces")
    print(f"   • Factor de frecuencia de tu número: {resultado['factor_frecuencia']:.2f}x")
    
    if resultado['factor_frecuencia'] > 1.5:
        print(f"   ⚠️  Este número sale MÁS que la media ({resultado['factor_frecuencia']:.1f}x más frecuente)")
    elif resultado['factor_frecuencia'] < 0.5 and resultado['veces_salido'] > 0:
        print(f"   ⚠️  Este número sale MENOS que la media")
    elif resultado['veces_salido'] == 0:
        print(f"   ⚠️  Este número NUNCA ha salido en los datos analizados")
    else:
        print(f"   ✓  Este número tiene una frecuencia normal")
    
    print("\n" + "=" * 70)
    print("⚠️  NOTA: La lotería es aleatoria. Los datos históricos NO predicen el futuro.")
    print("=" * 70)
    
    return resultado

# Programa principal
if __name__ == "__main__":
    print("\n🎰 CALCULADORA DE PROBABILIDAD DE LOTERÍA 🎰\n")
    
    while True:
        numero_input = input("\nIntroduce un número de lotería (5 dígitos, o 'salir' para terminar): ")
        
        if numero_input.lower() in ['salir', 'exit', 'q']:
            print("\n¡Hasta luego! 🍀")
            break
        
        # Validar input
        try:
            numero_validado = str(int(numero_input)).zfill(5)
            if int(numero_input) < 0 or int(numero_input) > 99999:
                print("❌ Error: El número debe estar entre 00000 y 99999")
                continue
        except ValueError:
            print("❌ Error: Introduce un número válido")
            continue
        
        # Calcular y mostrar resultados
        mostrar_resultados(numero_validado)
        
        # Preguntar si quiere analizar otro
        continuar = input("\n¿Quieres analizar otro número? (s/n): ")
        if continuar.lower() not in ['s', 'si', 'sí', 'y', 'yes']:
            print("\n¡Hasta luego! 🍀")
            break


🎰 CALCULADORA DE PROBABILIDAD DE LOTERÍA 🎰




Introduce un número de lotería (5 dígitos, o 'salir' para terminar):  76214


ANÁLISIS DE PROBABILIDAD PARA EL NÚMERO: 76214

📊 DATOS HISTÓRICOS:
   • El número 76214 ha salido: 0 veces
   • Total de sorteos analizados: 21684
   • Números únicos premiados: 19647

🎲 PROBABILIDADES:

   1. Probabilidad TEÓRICA (real entre todos los españoles):
      → 1 entre 100,000 números posibles
      → 0.00100%
      → Odds: 1:100,000

   2. Probabilidad HISTÓRICA (basada en los datos recogidos):
      → Basada en 21684 sorteos
      → 0.00000%

   3. Probabilidad entre NÚMEROS QUE HAN SALIDO:
      → Solo considerando los 19647 números que han sido premiados
      → 0.00000%

   📍 CENTENA (terminados en 214):
      • Ha salido: 0 veces
      • Probabilidad teórica: 0.100% (1 en 1,000)
      • Probabilidad histórica: 0.000%

   📍 DECENA (terminados en 14):
      • Ha salido: 2 veces
      • Probabilidad teórica: 1.00% (1 en 100)
      • Probabilidad histórica: 1.282%

   📍 REINTEGRO (terminados en 4):
      • Ha salido: 12 veces
      • Probabilidad teórica: 10.0% (1 en 10)
